# Hyperparameter Tuning using HyperDrive

Import Dependencies. In the cell below, import all the dependencies that I will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment

from azureml.core.dataset import Dataset

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive import choice, randint
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal

from azureml.widgets import RunDetails

## Dataset

Get data. In the cell below, I write code to access the data I will be using in this project.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'HyperTune'

experiment=Experiment(ws, experiment_name)

In [ ]:
found = False
key = "Employee Attrition"
description_text = "IBM HR Analytics Employee Attrition & Performance"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        url = "https://raw.githubusercontent.com/eljandoubi/Azure-Machine-Learning-Engineer/main/attrition-dataset.csv"
        dataset = Dataset.Tabular.from_delimited_files(url)        
        dataset = dataset.register(workspace=ws,
                                   name=key,description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
cluster_name = "automl_vs_hpyer"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target, using it!')
    
except ComputeTargetException:
    print('Creating a new compute target!')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',min_nodes=1, max_nodes=8)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)
 
cpu_cluster.get_status().serialize()

## Hyperdrive Configuration

Here I will set configuration for different parameters for our hyperdrive run.

I will use `RandomParameterSampling` as the Sampling method, `BanditPolicy` as the Termination policy, and `SKLearn estimator` with the Primary metric as `AUC_weighted`.

In [ ]:
# Creating an early termination policy.
early_termination_policy = BanditPolicy(slack_factor=0.1,evaluation_interval=3)

#Creating the different params that will be used during training
param_sampling = RandomParameterSampling({"--criterion": choice("gini", "entropy"),
                                          "--bootstrap": choice(True, False), 
                                          "--max_depth": randint(10)})

#Creating estimator and hyperdrive config
estimator = SKLearn(source_directory=".", compute_target=cpu_cluster, entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                         policy=early_termination_policy, 
                                         primary_metric_name="AUC_weighted", 
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                         max_total_runs=8, 
                                         max_concurrent_runs=2, 
                                         estimator=estimator)

In [ ]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

In the cell below, I use the `RunDetails` widget to show the different experiments.

In [ ]:

RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

In the cell below, I get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
 
print('Best Run Id: ', best_run.id)
print('AUC_weighted of Best Run is:', best_run_metrics['AUC_weighted'])
print('Parameter Values are:',best_run.get_details()['runDefinition']['arguments'])

In [ ]:
model = best_run.register_model(model_name='hyperdrive-model', 
                                model_path='outputs/model.joblib', 
                                tags={'Method':'Hyperdrive'}, 
                                properties={'AUC_weighted': best_run_metrics['AUC_weighted']})